In [2]:
from pytdx.hq import TdxHq_API
from pytdx.params import TDXParams
from datetime import datetime, timedelta
import pandas as pd
import polars as pl
import os
from tqdm import tqdm

HOST = '119.147.212.81'
PORT = 7709

API_TICK_SIZE = 2000

In [2]:
api = TdxHq_API()

In [3]:
df_cn_stks = pl.read_csv('stk_list/cn_list.csv')
df_cn_stks.head()

df_cn_stks['code'] = df_cn_stks['code'].apply(lambda x: str(x).zfill(6))
df_cn_stks.head()

symbol,name,tdx_api_mkt,code
str,str,i64,str
"""SZ002875""","""安奈儿""",0,"""002875"""
"""SZ002873""","""新天药业""",0,"""002873"""
"""SZ002870""","""香山股份""",0,"""002870"""
"""SZ002869""","""金溢科技""",0,"""002869"""
"""SZ002868""","""绿康生化""",0,"""002868"""


In [4]:
with api.connect(HOST, PORT):
    for i in tqdm(list(range(len(df_cn_stks[0:])))):
        mkt = df_cn_stks['tdx_api_mkt'][i]
        code = df_cn_stks['code'][i]
        
        # # manually set symbol
        # mkt = 1
        # code = '688076'

        data = api.get_security_bars(5, mkt, code, 1, 150)
        df_tmp = api.to_df(data)

        df = df_tmp[['open', 'close', 'high', 'low', 'vol', 'amount', 'datetime']]
        df = pl.DataFrame(df)
        df.to_parquet(f'cn_data_weekly_bar/data/{code}.parquet')

        # break

100%|██████████| 2024/2024 [02:49<00:00, 11.91it/s]


In [4]:
df = pl.read_parquet('cn_data_weekly_bar/data/002860.parquet')

In [5]:
dts = []
for dt_str in df['datetime']:
    dts.append(datetime.strptime(dt_str.split(' ')[0], '%Y-%m-%d'))

df['datetime'] = dts
df.head()

open,close,high,low,vol,amount,datetime
f64,f64,f64,f64,f64,f64,datetime[μs]
21.41,22.48,23.95,21.32,155485.0,3.49864736e8,2020-02-14 00:00:00
22.48,23.92,24.26,22.46,237629.0,5.60006528e8,2020-02-21 00:00:00
24.05,22.99,25.17,22.6,329196.0,7.8997568e8,2020-02-28 00:00:00
23.3,23.99,25.38,23.3,378570.0,9.26129408e8,2020-03-06 00:00:00
23.4,23.13,25.95,21.3,362561.0,8.81988864e8,2020-03-13 00:00:00


In [12]:
df['freq'] = ['1W'] * len(df)
df['symbol'] = ['002860'] * len(df)

In [13]:
df.head()

open,close,high,low,vol,amount,datetime,freq,symbol
f64,f64,f64,f64,f64,f64,datetime[μs],str,str
21.41,22.48,23.95,21.32,155485.0,3.49864736e8,2020-02-14 00:00:00,"""1w""","""002860"""
22.48,23.92,24.26,22.46,237629.0,5.60006528e8,2020-02-21 00:00:00,"""1w""","""002860"""
24.05,22.99,25.17,22.6,329196.0,7.8997568e8,2020-02-28 00:00:00,"""1w""","""002860"""
23.3,23.99,25.38,23.3,378570.0,9.26129408e8,2020-03-06 00:00:00,"""1w""","""002860"""
23.4,23.13,25.95,21.3,362561.0,8.81988864e8,2020-03-13 00:00:00,"""1w""","""002860"""


In [15]:
# 导出 Visual 数据文件
df.to_parquet(f'../analyzer.view.server/dump/cn_weekly/compute/002860.1W.parquet')